In [1]:
from Train_Test_Data import train_datasets, test_datasets
from Train_Test_Data import drug_list

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.data import DataLoader
from torch.utils.data import TensorDataset
import random
import numpy as np

#call pytorch lightning functions
import pytorch_lightning as pl
from pytorch_lightning import Callback
from pytorch_lightning import Trainer, seed_everything

from torch_geometric.data import DataLoader as GeoDataLoader

from tqdm import tqdm


Step 0: Download and Preprocess
          Oxaliplatin  Ulixertinib  Fulvestrant  Uprosertib  Dactinomycin  \
22RV1        3.081337     2.803572     4.227332    1.133014     -4.923521   
23132-87     4.391634     2.526244     3.697545    0.188871     -4.530814   
42-MG-BA     4.129735     3.147708     4.045351    1.682197     -4.231256   
5637         4.391109     2.884360     3.535578    4.453362     -4.262836   
639-V        3.253815     2.546112     3.550254    4.297324     -4.760262   
...               ...          ...          ...         ...           ...   
YAPC         5.245318     4.309617     4.761417    5.532699     -2.255783   
YH-13        3.945620     3.377953     3.812616    2.172373     -3.734196   
YT           0.461330     1.745128     2.024318    4.395238     -5.679264   
ZR-75-30     7.154014     6.182055     5.587853    3.443328     -0.055238   
huH-1        6.006727     4.090465     4.028713    3.325009     -0.850808   

          Docetaxel  Camptothecin  5-Fluor

/sybig/home/tmu/Schreibtisch/Bachelor-Thesis/Networks/Train_Test_Data.py:49: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  labels_long = labels_df.stack(dropna=False).reset_index()
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!



--- Fold 2 ---
Start building Train Dataset
536
Start building Test Dataset
268

--- Fold 3 ---
Start building Train Dataset
536
Start building Test Dataset
268


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, TopKPooling, global_max_pool as gmp, global_mean_pool as gap

class GNNEncoder(nn.Module):
    def __init__(self, feature_size=3, embedding_size=512, output_dim=256):
        super(GNNEncoder, self).__init__()

        # Block 1
        self.conv1 = GATConv(feature_size, embedding_size, heads=3, dropout=0.6)
        self.head_transform1 = nn.Linear(embedding_size * 3, embedding_size)
        self.bn1 = nn.BatchNorm1d(embedding_size)
        self.pool1 = TopKPooling(embedding_size, ratio=0.8)

        # Block 2
        self.conv2 = GATConv(embedding_size, embedding_size, heads=3, dropout=0.6)
        self.head_transform2 = nn.Linear(embedding_size * 3, embedding_size)
        self.bn2 = nn.BatchNorm1d(embedding_size)
        self.pool2 = TopKPooling(embedding_size, ratio=0.5)

        # Final projection
        self.linear1 = nn.Linear(embedding_size * 2, 512)
        self.linear2 = nn.Linear(512, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Block 1
        x = self.conv1(x, edge_index)
        x = F.relu(self.head_transform1(x))
        x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        # Block 2
        x = self.conv2(x, edge_index)
        x = F.relu(self.head_transform2(x))
        x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        # Combine
        x = x1 + x2

        # Final layers
        x = self.linear1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        graph_embedding = self.linear2(x)

        return graph_embedding

In [3]:
class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_test_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
class tugda_mtl_gnn(pl.LightningModule):
    def __init__(self, params, train_dataset, y_train, test_dataset, y_test):
        super(tugda_mtl_gnn, self).__init__()
        
        # Hyperparameters
        self.learning_rate = params['lr']
        self.batch_size = params['bs']
        self.mu = params['mu']
        self.lambda_ = params['lambda_']
        self.gamma = params['gamma']
        self.num_tasks = params['num_tasks']
        self.passes = params['passes']

        # Data
        self.train_data = train_dataset
        self.y_train = y_train
        self.test_data = test_dataset
        self.y_test = y_test
        
        # GNN Encoder (your existing implementation)
        self.gnn_encoder = GNNEncoder(
            feature_size=params.get('feature_size', 3),
            embedding_size=params.get('embedding_size', 128),
            output_dim=params.get('gnn_output_dim', 64)
        )

        # Projection layer
        projection = [
            nn.Linear(params.get('gnn_output_dim', 256), params['hidden_units_1']),
            nn.ReLU()
        ]
        self.projection = nn.Sequential(*projection)
        
        # Latent basis
        latent_basis = [
            nn.Linear(params['hidden_units_1'], params['latent_space']),
            nn.Dropout(p=params['dropout']),
            nn.ReLU()
        ]
        self.latent_basis = nn.Sequential(*latent_basis)
        
        # Task-specific weights
        self.S = nn.Linear(params['latent_space'], self.num_tasks)
        
        # Decoder weights
        A = [nn.Linear(self.num_tasks, params['latent_space']), nn.ReLU()]
        self.A = nn.Sequential(*A)
        
        # Uncertainty (aleatoric)
        self.log_vars = torch.zeros(self.num_tasks, requires_grad=True, device=device)
        
    def forward(self, data):
        # Process graph data through GNN encoder
        graph_embedding = self.gnn_encoder(data)
        
        # Project to hidden dimension
        x = self.projection(graph_embedding)
        
        # Continue with original TUGDA architecture
        h = self.latent_basis(x)
        preds = self.S(h)
        h_hat = self.A(preds)
        return preds, h, h_hat

    def prepare_data(self):
        self.train_dataset = self.train_data
        self.test_dataset = self.test_data

    def train_dataloader(self):
        from torch_geometric.loader import DataLoader
        return DataLoader(self.train_dataset, 
                        batch_size=self.batch_size,
                        shuffle=True,
                        num_workers=0)
    
    def test_dataloader(self):
        from torch_geometric.loader import DataLoader
        return DataLoader(self.test_dataset,
                        batch_size=len(self.test_dataset),
                        shuffle=False,
                        num_workers=0)
    
    def configure_optimizers(self):
        params = ([p for p in self.parameters()] + [self.log_vars])
        optimizer = torch.optim.Adagrad(params, lr=self.learning_rate)
        return optimizer
    
    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_closure):
        """
        Proper implementation of optimizer_step that handles the closure
        """
        # Call the closure to compute the loss and gradients
        optimizer_closure()
        
        # Perform the actual optimization step
        optimizer.step()
        
        # Zero gradients
        optimizer.zero_grad()
        
    def mse_ignore_nan(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='none')
        per_task_loss = torch.zeros(labels.size(1), device=device)

        for k in range(labels.size(1)):
            precision = torch.exp(-self.log_vars[k])
            diff = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            per_task_loss[k] = torch.mean(precision * diff + self.log_vars[k])

        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss
    
    def mse_ignore_nan_test(self, preds, labels):
        mse_loss = torch.nn.MSELoss(reduction='mean')
        per_task_loss = torch.zeros(labels.size(1), device=device)
        
        for k in range(labels.size(1)):
            per_task_loss[k] = mse_loss(preds[~torch.isnan(labels[:,k]), k], labels[~torch.isnan(labels[:,k]), k])
            
        return torch.mean(per_task_loss[~torch.isnan(per_task_loss)]), per_task_loss 
    
    def MSE_loss(self, x, x_hat):
        mse_loss = torch.nn.MSELoss()
        return mse_loss(x, x_hat)
    
    def forward_pass(self, batch, batch_idx):
        # print("Type of batch:", type(batch))
        # print("Length of batch:", len(batch))  # Wie viele Cell Lines im Batch?
        # print("Type of first element:", type(batch[0]))
        # print("First element keys:", batch[0][0].keys())  # Zeigt an, welche Attribute der Graph hat

 
        cell_line_graphs = batch[0]  # Liste von Drug-Graphen
        labels = batch[1]            # shape: [num_drugs]
        labels = labels.squeeze(1)

        # print(type(cell_line_graphs)) 
        # print(len(cell_line_graphs)) 
        # print(labels)
        # print(labels.shape)

        all_embeddings = []

        for graph in cell_line_graphs:
            # GNN Encoder aufrufen
            graph_emb = self.gnn_encoder(graph)  # shape: [1, output_dim]
            all_embeddings.append(graph_emb)

        # Staple alle Embeddings
        graph_embeddings = torch.cat(all_embeddings, dim=0)  # shape: [num_drugs, output_dim]

        # Projektion in latenten Raum
        h = self.latent_basis(graph_embeddings)

        # Task-spezifische Vorhersage
        preds = self.S(h).squeeze(1)  # shape: [num_cell_lines, num_tasks]

        # Loss-Berechnung
        local_loss, task_loss = self.mse_ignore_nan(preds, labels)

        recon_loss = self.gamma * self.MSE_loss(h, ...)  # optional

        # Regularisierungen
        a = 1 + torch.sum(torch.abs(self.A[0].weight.T), 1)
        loss_weight = (a[~torch.isnan(task_loss)]) * task_loss[~torch.isnan(task_loss)]
        loss_weight = torch.sum(loss_weight)
        l1_S = self.mu * self.S.weight.norm(1)
        L = self.latent_basis[0].weight.norm(2) + self.gnn_encoder.norm().norm(2)
        l2_L = self.lambda_ * L

        total_loss = loss_weight + recon_loss + l1_S + l2_L

        return total_loss, task_loss
    def training_step(self, batch, batch_idx):
        # batch is a PyG Data object, no need to unpack
        loss, task_loss = self.forward_pass(batch, batch_idx)
        self.log('train_loss', loss)
        return loss

    
    def test_step(self, test_batch, batch_idx):
        # Enable dropouts in GNN encoder
        for module in self.gnn_encoder.modules():
            if isinstance(module, nn.Dropout):
                module.train()

        y = test_batch.y  # shape: [batch_size, num_tasks]
        preds_simulation = torch.zeros(y.size(0), y.size(1), self.passes, device=self.device)

        for simulation in range(self.passes):
            seed = simulation
            pl.seed_everything(seed)
            preds, _, _ = self.forward(test_batch)
            preds_simulation[:, :, simulation] = preds

        preds_mean = torch.mean(preds_simulation, dim=2)
        loss, task_losses_per_class = self.mse_ignore_nan_test(preds_mean, y)

        # Disable dropouts
        self.eval()

        return {
            'test_loss': loss,
            'test_task_losses_per_class': task_losses_per_class.detach().cpu().numpy(),
            'test_preds': preds_mean.detach().cpu().numpy(),
        }

In [7]:
#best set of hyperparamters found on this dataset setting (GDSC)
net_params = {
 #tunned hyperparameters
 'hidden_units_1': 512,
 'latent_space': 256,
 'lr': 0.001,
 'dropout': 0.1,
 'mu': 0.01,
 'lambda_': 0.001,
 'gamma': 0.0001,
 'bs': 8,
 'passes': 10,
 'num_tasks': 200,
 'epochs': 10}

In [24]:
print(train_datasets[0][0])

([Data(x=[1785, 3], edge_index=[2, 47063], y=[1], drug='Oxaliplatin', cell_line='22RV1', nodes=[1785]), Data(x=[1783, 3], edge_index=[2, 48145], y=[1], drug='Ulixertinib', cell_line='22RV1', nodes=[1783]), Data(x=[1925, 3], edge_index=[2, 76685], y=[1], drug='Fulvestrant', cell_line='22RV1', nodes=[1925]), Data(x=[1776, 3], edge_index=[2, 47986], y=[1], drug='Uprosertib', cell_line='22RV1', nodes=[1776]), Data(x=[1841, 3], edge_index=[2, 56445], y=[1], drug='Dactinomycin', cell_line='22RV1', nodes=[1841]), Data(x=[1754, 3], edge_index=[2, 45226], y=[1], drug='Docetaxel', cell_line='22RV1', nodes=[1754]), Data(x=[1827, 3], edge_index=[2, 52461], y=[1], drug='Camptothecin', cell_line='22RV1', nodes=[1827]), Data(x=[1703, 3], edge_index=[2, 35167], y=[1], drug='5-Fluorouracil', cell_line='22RV1', nodes=[1703]), Data(x=[1849, 3], edge_index=[2, 59683], y=[1], drug='Afatinib', cell_line='22RV1', nodes=[1849]), Data(x=[1669, 3], edge_index=[2, 32873], y=[1], drug='Taselisib', cell_line='22RV

In [8]:
pcorr_list = []

metrics_callback = MetricsCallback()

for k in range(1, 4):  # 3-fold
    
    print(f"\n--- Fold {k} ---")
    
    # Original-Daten laden
    full_train_dataset = train_datasets[k - 1]
    full_test_dataset = test_datasets[k - 1]

    full_y_train = full_train_dataset.labels_df.values
    full_y_test = full_test_dataset.labels_df.values

    X_train = full_train_dataset
    X_test = full_test_dataset
    y_train = full_y_train
    y_test = full_y_test

    # Trainer erstellen
    trainer = pl.Trainer(
        max_epochs=net_params['epochs'],
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        callbacks=[metrics_callback],
        deterministic=True,
        # reload_dataloaders_every_epoch=True
    )

    # Seeds setzen
    seed = 42
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(seed)

    # Modell erstellen
    model = tugda_mtl_gnn(net_params, X_train, y_train, X_test, y_test)

    # Training starten
    trainer.fit(model)

    # Testen
    results = trainer.test(model)

    # Fehler pro Drug speichern
    error_per_task = results[0]['test_task_losses_per_class']
    error_mtl_nn_results = np.concatenate((
        np.array(drug_list, ndmin=2).T,
        np.array(error_per_task, ndmin=2).T
    ), axis=1)

    print("Error per task:")
    print(error_mtl_nn_results)

INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/sybig/home/tmu/miniconda3/envs/node/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type       | Params | Mode 
----------------------------------------------------
0 | gnn_en


--- Fold 1 ---
Epoch 0:   0%|          | 0/67 [00:00<?, ?it/s] Type of batch: <class 'list'>
Length of batch: 2
Type of first element: <class 'list'>
First element keys: ['nodes', 'ptr', 'cell_line', 'drug', 'y', 'x', 'edge_index', 'batch']
<class 'list'>
200
tensor([[ 4.5958,  1.4500,  4.2829,  ..., -0.3595,  1.7398,  7.0242],
        [ 4.9857,  3.5065,  4.8551,  ...,  5.8730,  2.7540,  7.4831],
        [ 2.5521,  0.8855,  3.3202,  ...,  2.7703, -2.6321,  6.9303],
        ...,
        [ 3.8278,  1.7202,  3.7536,  ..., -0.7913,  2.5872,  6.7696],
        [ 4.5319,  2.6819,  2.8412,  ...,  2.2491,  2.5580,  5.9608],
        [ 5.8677,  3.8628,  4.8139,  ...,  3.2862,  5.2424,  7.8333]])
torch.Size([8, 200])


INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined